In [1]:
# Import
from scipy.integrate import odeint
import numpy as np
from numpy import genfromtxt

In [2]:
# Constants
alpha = 2e-6
cm = 45
cM = 55
tw = 5
mde = 30
mdry = 2000
mp = 5000
rpm = 6600000
mu = 398600.4418e9
J2 = 1.08262668e-3
req = 6378137
Isp = 340
g0 = 9.80665
day = 86400
year = 365.25

In [3]:
debris = genfromtxt('gtoc9_debris.csv', delimiter=',')
debris = debris[1:,:] # Get rid of header
# id,epoch [mjd2000],a [m],e,i [rad],Omega [rad],omega [rad],M [rad]
print(debris[49,:])

[4.90000000e+01 2.19192589e+04 7.16362984e+06 6.00222048e-03
 1.69404266e+00 2.88962130e-01 1.06641880e+00 4.40355335e+00]


In [4]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
# Definition of integration
def integrateEOM(X,t):
    mu = 398600.4418e9
    J2 = 1.08262668e-3
    req = 6378137
    r = np.linalg.norm(X[0:3])
    x = X[0]
    y = X[1]
    z = X[2]
    dx2dt2 = -mu*x/r**3*(1+3/2*J2*(req/r)**2*(1-5*z**2/r**2))
    dy2dt2 = -mu*y/r**3*(1+3/2*J2*(req/r)**2*(1-5*z**2/r**2))
    dz2dt2 = -mu*z/r**3*(1+3/2*J2*(req/r)**2*(3-5*z**2/r**2))
#     print(np.linalg.norm([x,y,z]))
    dXdt = np.array([X[3],X[4],X[5],dx2dt2,dy2dt2,dz2dt2])
    return dXdt

In [5]:
# http://murison.alpheratz.net/dynamics/twobody/KeplerIterations_summary.pdf
def KeplersProblem(e,M,tol):
    Mnorm = np.mod(M,2*np.pi)
    t34 = e**2
    t35 = e*t34
    t33 = np.cos(M)
    E0 = Mnorm+(-1/2*t35+e+(t34+3/2*t33*t35)*t33)*np.sin(M)
    dE = tol + 1
    count = 0
    while dE > tol:
        t1 = np.cos(E0)
        t2 = -1+e*t1
        t3 = np.sin(E0)
        t4 = e*t3
        t5 = -E0+t4+Mnorm
        t6 = t5/(1/2*t5*t4/t2+t2)
        Eadjust = t5/((1/2*t3-1/6*t1*t6)*e*t6+t2)
        E = E0-Eadjust
        dE = np.abs(E-E0)
        E0 = E
        count = count +1
        if count > 100:
            print('Exceeded Iterations for finding E')
            break
    # print(count)
    return E

In [6]:
# for ndx in range(np.size(debris)-1):
ndx = 86
t = 2.1990023072652064E+04*day
oe = debris[ndx,:]
id = oe[0]
t0 = oe[1]*day
a = oe[2]
e = oe[3]
i = oe[4]
Omega0 = oe[5]
omega0 = oe[6]
M0 = oe[7]
# print(id)
# print(a)
# print(e)
# print(i)
# print(Omega0)
# print(omega0)
# print(M0)

# Equations
# Compute Keplerian
n = (mu/a**3)**(1/2)
p = a*(1-e**2)
Omegadt = -3/2*J2*(req/p)**2*n*np.cos(i)
omegadt = 3/4*J2*(req/p)**2*n*(5*np.cos(i)**2-1)
Omega = Omegadt*(t-t0)+Omega0
omega = omegadt*(t-t0)+omega0
M = n*(t-t0)+M0

# Compute Cartesian
# E = M+e*np.sin(E)
E = KeplersProblem(e,M,1e-14) # This seems to work
theta = np.arctan(np.tan(E/2)/((1-e)/(1+e))**(1/2))*2
gamma = np.arctan((e*np.sin(theta))/(1+e*np.cos(theta)))
r = a*(1-e**2)/(1+e*np.cos(theta))
v = (2*mu/r-mu/a)**(1/2)
x = r*(np.cos(theta+omega)*np.cos(Omega)-np.sin(theta+omega)*np.cos(i)*np.sin(Omega))
y = r*(np.cos(theta+omega)*np.sin(Omega)+np.sin(theta+omega)*np.cos(i)*np.cos(Omega))
z = r*(np.sin(theta+omega)*np.sin(i))
vx = v*(-np.sin(theta+omega-gamma)*np.cos(Omega)-np.cos(theta+omega-gamma)*np.cos(i)*np.sin(Omega))
vy = v*(-np.sin(theta+omega-gamma)*np.sin(Omega)+np.cos(theta+omega-gamma)*np.cos(i)*np.cos(Omega))
vz = v*(np.cos(theta+omega-gamma)*np.sin(i))
X0 = np.array([x,y,z,vx,vy,vz])
print(X0)


[ 1.97624770e+06  3.23277426e+06  6.12078125e+06 -1.05223268e+03
 -6.35412773e+03  3.69086197e+03]


In [7]:
# Debris test
# id, epoch [mjd2000], rx, ry, rz, vx, vy, vz [m and m/s]
# 49 2.0376643799980138E+04 -1.8213529728684309E+06 -1.4807329391408276E+06 -6.8134374313306045E+06 -6.8037943427245509E+03 -1.9247047977499390E+03 2.2327104878827681E+03
# 86 2.1990023072652064E+04 1.9762477001880330E+06 3.2327742582852468E+06 6.1207812503049523E+06 -1.0522326782837285E+03 -6.3541277289958516E+03 3.6908619692955722E+03
# 14 2.1952676528942095E+04 1.7185904037106584E+05 -4.1441151415295480E+06 5.8389232250198871E+06 -1.8871362747878838E+03 5.8904780272571370E+03 4.1904439853599897E+03
# 7 2.1864963278453706E+04 -5.5164765114898002E+06 5.8743104578474001E+05 4.3377730408311943E+06 4.6845232520294676E+03 8.0603029187888046E+02 5.8434965437240890E+03
# 56 2.2660259705606291E+04 -1.7831477917911313E+06 -2.8275671044597263E+06 -6.4258672727777855E+06 -5.3270335613483239E+03 -3.9492437672484357E+03 3.1942723766509312E+03
# 53 2.1627743853536944E+04 6.9698288365489021E+06 9.9676946205251070E+05 -9.9340413272313960E+05 -9.9709822664859371E+02 -1.3115239203630188E+03 -7.3091438055596927E+03
# 45 2.0750025785868245E+04 9.6830390395430545E+05 5.4584968344897451E+05 -7.1570787886344362E+06 9.1805157584077756E+02 -7.3271948600165633E+03 -3.8770692668554420E+02
# 105 2.2405715549506494E+04 1.4061011968226654E+06 1.9570602092289515E+06 -6.7479717887752764E+06 6.4829677314747059E+03 2.9716864866252290E+03 2.1718560743681855E+03
# 110 2.2048799818887353E+04 -1.1642908668323851E+05 -3.2660594455643641E+06 6.1774193832164854E+06 -2.4225301727500273E+03 -6.3422926784303236E+03 -3.3701590353910178E+03
# 114 2.2122223389120474E+04 -2.5409506692402065E+05 -5.4245997422103481E+06 4.6604288439709237E+06 -1.7426726246735980E+03 -4.7346145547534452E+03 -5.4682016415434791E+03
# 104 2.1442582122029275E+04 -4.3618158503376273E+06 2.6733686402653605E+06 4.9680798581335545E+06 5.0949019802617413E+03 -1.4270942815747119E+03 5.2869939806983302E+03
# 121 2.2452262052279784E+04 8.9715177926246217E+05 -5.1586529262571561E+05 6.9585562469750000E+06 -2.8571286171193715E+03 -6.9680000022658087E+03 -1.4732007818800253E+02
# 100 2.2810775926435661E+04 2.8371477924629459E+06 -2.6013871747195865E+06 5.9308698328733454E+06 -5.8951322977971022E+03 2.5739533190329535E+03 3.9677169412219400E+03
# 51 2.2292431022220873E+04 -3.7888830414882866E+06 4.0485188886715053E+05 6.1960280816609599E+06 6.2748699833859391E+03 1.1386417360361249E+03 3.7587414423603477E+03
# 99 2.0645743323503342E+04 2.8397461995777842E+06 -3.2779148626603945E+06 5.7778500047742808E+06 -5.0999378696882559E+03 3.1788363279180699E+03 4.3004246278860319E+03
# 46 2.0257814779210727E+04 7.8669685224297224E+04 -1.5557339750105096E+06 -7.0042518321334701E+06 -5.3011523238455247E+03 -5.1260793617466979E+03 1.0736165247161894E+03
# 3 2.0242872337220160E+04 -5.4726816866906844E+06 -4.1391068092223210E+06 1.9610815813742168E+06 1.0769675403280298E+03 2.0127936904808794E+03 7.1021053839580254E+03
# 18 2.0022366191389960E+04 2.3246283473022655E+06 -6.7655375873452500E+06 6.0342693619994633E+05 -1.2454210930616837E+03 2.1487742221561962E+02 7.3097237586399460E+03
# 23 2.1918873465584344E+04 2.2925670047330847E+06 -3.0630104367745770E+06 -5.9902829424053151E+06 1.9968853959851417E+03 -6.1689668995932770E+03 3.7978380882976562E+03
# 15 2.1173930593818935E+04 -3.1064877245512139E+05 -4.3572395846349644E+06 -5.6601470666763419E+06 -2.2867074000744888E+03 -5.5590750892300630E+03 4.4580818398481561E+03
# 29 2.1582926756788325E+04 -3.9838342948962201E+05 2.6888655777318943E+06 6.7027856079840586E+06 2.0612672638413387E+03 6.6621343901345899E+03 -2.5134990950241499E+03
# 60 2.2466022245161483E+04 6.4163611165560754E+06 3.4680930006649694E+04 3.2028174583411096E+06 -3.2728765105376065E+03 -1.3189385402935482E+03 6.5675144033470206E+03
# 112 2.2093684278253535E+04 -2.2521644842390483E+06 5.9670643768276535E+06 3.1918677646631016E+06 -2.0463224975262619E+02 3.5071357057441887E+03 -6.5965553657572873E+03
# 16 2.2108415470478674E+04 -6.0813734810287273E+06 3.4861449684904781E+06 -9.0421178978705499E+05 -3.3743976733393674E+02 1.4514819204110593E+03 7.3721274214127925E+03
# 66 2.1325717218675669E+04 8.2023638491693651E+05 6.5451474667701719E+05 7.0334058345152335E+06 6.2304303164865905E+03 -4.1926098401696618E+03 -3.2429843868925650E+02
# 53 2.2146551393980229E+04 -1.1538738426071862E+06 -1.2227819738105009E+06 6.8500735702787545E+06 -7.3650254719288305E+03 -7.4410232301473798E+02 -1.4994188044403618E+03
# 17 2.0059334633886596E+04 2.2456887915754477E+06 6.3892664338445095E+06 2.2760420583209349E+06 1.8482323514142493E+03 1.8984531045461115E+03 -6.9987637306847109E+03
# 97 2.1523346804783378E+04 -5.3143494592636377E+06 4.5822433590138610E+06 9.3357484696642752E+05 1.3925884723006227E+03 1.0758158438388750E+02 7.3786928368068175E+03
# 35 2.0187498695430622E+04 5.8428663342795819E+06 -3.7260323823948987E+06 -1.9168601227906104E+06 -2.1922321384541133E+03 3.3404993575208846E+02 -7.1031427531574336E+03
# 73 2.2414322884597008E+04 1.6398277748807850E+06 -8.6532899287303642E+05 6.9263846855307342E+06 -7.1940052031752221E+03 -8.3365863578583549E+02 1.6093333824461351E+03

In [8]:

# Integrate Equations of Motion
t0 = 2.3567000000000000E+04*day
t = 2.3576750000000000E+04*day
numStep = 842400/21600+1
x = -9.0656779992979474E+05
y = -4.8397431127596954E+06
z = -5.0408120071376814E+06
vx = -7.6805804020022015E+02
vy = 5.4710987127502622E+03
vz = -5.1022193482389539E+03
X0 = np.array([x,y,z,vx,vy,vz])
r = np.linalg.norm(X0[0:3])
tspan = np.linspace(t0,t,numStep)-t0
X = odeint(integrateEOM,X0,tspan,mxstep=10000000)
# print([0,(t-t0)])
print(X)
# print(np.linalg.norm(X0[0:3]))
# print(np.linalg.norm(X[0,0:3]))
# print(np.linalg.norm(X[1,0:3]))
print(tspan/day)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  


[[-9.06567800e+05 -4.83974311e+06 -5.04081201e+06 -7.68058040e+02
   5.47109871e+03 -5.10221935e+03]
 [ 1.09204810e+06 -1.97208817e+06  6.68522547e+06 -4.08366663e+02
  -7.20827263e+03 -2.05816577e+03]
 [-2.53387918e+05  6.83539253e+06 -1.70135543e+06  1.20112168e+03
   1.84292903e+03  7.19403422e+03]
 [-8.16403006e+05 -4.96029747e+06 -4.93791204e+06 -8.69118332e+02
   5.34788359e+03 -5.21574846e+03]
 [ 1.12735897e+06 -1.81107452e+06  6.72475912e+06 -2.74420855e+02
  -7.25599929e+03 -1.90689336e+03]
 [-3.79536833e+05  6.79257325e+06 -1.84369657e+06  1.16486352e+03
   2.01479475e+03  7.15362014e+03]
 [-7.22607214e+05 -5.07629393e+06 -4.83347720e+06 -9.66103253e+02
   5.22120946e+03 -5.32621728e+03]
 [ 1.15752652e+06 -1.64958922e+06  6.76102917e+06 -1.39227879e+02
  -7.29783287e+03 -1.75574134e+03]
 [-5.04028131e+05  6.74486332e+06 -1.98391805e+06  1.12329791e+03
   2.18362535e+03  7.11040465e+03]
 [-6.25439853e+05 -5.18722881e+06 -4.72802218e+06 -1.05885316e+03
   5.09175149e+03 -5.4331

In [10]:
# Integration Test
# epoch [mjd2000], rx, ry, rz, vx, vy, vz [m and m/s]
# 2.3567000000000000E+04 -9.0656779992979474E+05 -4.8397431127596954E+06 -5.0408120071376814E+06 -7.6805804020022015E+02 5.4710987127502622E+03 -5.1022193482389539E+03
# 2.3567250000000000E+04 1.0920601198177545E+06 -1.9719312043406873E+06 6.6852892646843726E+06 -4.0833841182019057E+02 -7.2083121930960815E+03 -2.0579927050199744E+03
# 2.3567500000000000E+04 -2.5351548984495568E+05 6.8352157692217119E+06 -1.7021213695013500E+06 1.2010894720186029E+03 1.8437525627076000E+03 7.1938182245087291E+03
# 2.3567750000000000E+04 -8.1623767555577948E+05 -4.9613406263258317E+06 -4.9369182195369545E+06 -8.6929599260473924E+02 5.3467906956042934E+03 -5.2168278978595226E+03
# 2.3568000000000000E+04 1.1274400939171189E+06 -1.8090272463331309E+06 6.7253204383930536E+06 -2.7406087103360500E+02 -7.2565646678370495E+03 -1.9047360134927003E+03
# 2.3568250000000000E+04 -3.7998608548162162E+05 6.7918240787807805E+06 -1.8464586075683380E+06 1.1646945577493977E+03 2.0177712316921316E+03 7.1527926969920372E+03
# 2.3568500000000000E+04 -7.2211356737964950E+05 -5.0789988339745114E+06 -4.8307543769701021E+06 -9.6652335690875759E+02 5.2182297923030628E+03 -5.3290426157189049E+03
# 2.3568750000000000E+04 1.1576255544683798E+06 -1.6447120269365069E+06 6.7622408178596273E+06 -1.3835360689783812E+02 -7.2990559508863171E+03 -1.7506091567220674E+03
# 2.3569000000000000E+04 -5.0498042397399090E+05 6.7430523928129701E+06 -1.9899504564349221E+06 1.1228082590823278E+03 2.1901257215015030E+03 7.1084607211957000E+03
# 2.3569250000000000E+04 -6.2433826007295109E+05 -5.1925782474316098E+06 -4.7223693379975623E+06 -1.0595912553244107E+03 5.0855685658895218E+03 -5.4388111247683701E+03
# 2.3569500000000000E+04 1.1825698911360730E+06 -1.4791815007045981E+06 6.7960334805398947E+06 -1.4225178065988426E+00 -7.3357367175292738E+03 -1.5956826075898455E+03
# 2.3569750000000000E+04 -6.2830798680215038E+05 6.6889592573358798E+06 -2.1325306229242897E+06 1.0754939271713874E+03 2.3606105672199778E+03 7.0608433885153981E+03
# 2.3570000000000000E+04 -5.2305950896485243E+05 -5.3019447009006450E+06 -4.6118129820265360E+06 -1.1483564390614936E+03 4.9489649882690092E+03 -5.5460821394031354E+03
# 2.3570250000000000E+04 1.2022336564404522E+06 -1.3126331936533761E+06 6.8266829500473849E+06 1.3652486106940441E+02 -7.3665651850870017E+03 -1.4400271715157978E+03
# 2.3570500000000000E+04 -7.4978050790048926E+05 6.6296101809435673E+06 -2.2741332309912886E+06 1.0228223590408763E+03 2.5290228122775306E+03 7.0099633411215573E+03
# 2.3570750000000000E+04 -4.1842984289148438E+05 -5.4069695293267556E+06 -4.4991361869064141E+06 -1.2326814842970061E+03 4.8085822335528637E+03 -5.6508054918370754E+03
# 2.3571000000000000E+04 1.2165846178750116E+06 -1.1452659404237082E+06 6.8541752026790688E+06 2.7527963182732657E+02 -7.3915071650392765E+03 -1.2837139565510861E+03
# 2.3571250000000000E+04 -8.6921222673635150E+05 6.5650775477192309E+06 -2.4146928527170066E+06 9.6487169865384976E+02 2.6951622816092349E+03 6.9558447587366227E+03
# 2.3571500000000000E+04 -3.1060636065291107E+05 -5.5075297393781990E+06 -4.3843908060504533E+06 -1.3124349806059329E+03 4.6645884641336452E+03 -5.7529321580860560E+03
# 2.3571750000000000E+04 1.2255978004072069E+06 -9.7727961979423999E+05 6.8784976732967310E+06 4.1463180945464859E+02 -7.4105361053653487E+03 -1.1268143434972103E+03
# 2.3572000000000000E+04 -9.8642013801807712E+05 6.4954405209128922E+06 -2.5541445390472459E+06 9.0172732793921455E+02 2.8588318507270324E+03 6.8985133445744050E+03
# 2.3572250000000000E+04 -1.9975052134008915E+05 -5.6035081729352772E+06 -4.2676296450699540E+06 -1.3874917115777894E+03 4.5171566097350496E+03 -5.8524142835584589E+03
# 2.3572500000000000E+04 1.2292555193111568E+06 -8.0887488885153818E+05 6.8996392605740670E+06 5.5437059869690177E+02 -7.4236331230733367E+03 -9.6939995600751456E+02
# 2.3572750000000000E+04 -1.1012242376995557E+06 6.4207849375020703E+06 -2.6924238502972671E+06 8.3348174794643717E+02 3.0198377103695939E+03 6.8379963104462686E+03
# 2.3573000000000000E+04 -8.6027928807004937E+04 -5.6947936628594799E+06 -4.1489064379318329E+06 -1.4577328273483336E+03 4.3664641397176110E+03 -5.9492052082253122E+03
# 2.3573250000000000E+04 1.2275474032923782E+06 -6.4025291618345247E+05 6.9175903316033296E+06 6.9428467182733573E+02 -7.4307870268874831E+03 -8.1154263069013518E+02
# 2.3573500000000000E+04 -1.2134477649446905E+06 6.3412031937878430E+06 -2.8294668863860792E+06 7.6023445031708798E+02 3.1779896263379746E+03 6.7743223610592013E+03
# 2.3573750000000000E+04 3.0391889429417894E+04 -5.7812811808310254E+06 -4.0282758226464218E+06 -1.5230460087856643E+03 4.2126928289273774E+03 -6.0432594913463199E+03
# 2.3574000000000000E+04 1.2204704078796285E+06 -4.7161511446174904E+05 6.9323427258610269E+06 8.3416244697522700E+02 -7.4319943300747427E+03 -6.5331438722877203E+02
# 2.3574250000000000E+04 -1.3229174397221520E+06 6.2567941221932806E+06 -2.9652103167638117E+06 6.8209177927958910E+02 3.3331011941370343E+03 6.7075216775361214E+03
# 2.3574500000000000E+04 1.4933570856491048E+05 -5.8628719770361800E+06 -3.9057933165170159E+06 -1.5833256230806307E+03 4.0560285174125388E+03 -6.1345329357100973E+03
# 2.3574750000000000E+04 1.2080288190691883E+06 -3.0316287274664378E+05 6.9438897585359234E+06 9.7379236667631278E+02 -7.4272592534085643E+03 -4.9478739850110901E+02
# 2.3575000000000000E+04 -1.4294636957144595E+06 6.1676628594177263E+06 -3.0995914100560336E+06 5.9916678437606538E+02 3.4849900881131512E+03 6.6376259001628432E+03
# 2.3575250000000000E+04 2.7062683786851529E+05 -5.9394737115388345E+06 -3.7815152909266436E+06 -1.6384728705029850E+03 3.8966608642817810E+03 -6.2229826113910985E+03
# 2.3575500000000000E+04 1.1902342472198480E+06 -1.3509728886838377E+05 6.9522262232182948E+06 1.1129631763049404E+03 -7.4165937182734415E+03 -3.3603396069923389E+02
# 2.3575750000000000E+04 -1.5329209082240660E+06 6.0739207061384087E+06 -3.2325480633862871E+06 5.1157906415329342E+02 3.6334783047226779E+03 6.5646681103919018E+03
# 2.3576000000000000E+04 3.9408535671821644E+05 -6.0110005771390097E+06 -3.6554989456896358E+06 -1.6883959220939419E+03 3.7347830960456608E+03 -6.3085668789837246E+03
# 2.3576250000000000E+04 1.1671056112084414E+06 3.2381097767744042E+04 6.9573483939502314E+06 1.2514642020497129E+03 -7.4000173299258040E+03 -1.7712646344650310E+02
# 2.3576500000000000E+04 -1.6331276167697462E+06 5.9756849784499155E+06 -3.3640188313657460E+06 4.1945460105847394E+02 3.7783923996055505E+03 6.4886828121249773E+03
# 2.3576750000000000E+04 5.1952835499668121E+05 -6.0773734135455815E+06 -3.5278022829865497E+06 -1.7330100480792596E+03 3.5705917497814453E+03 -6.3912454122829995E+03